# Bulk RNA-Seq Analysis Training Demo

## Overview

This short tutorial demonstrates how to run an RNA-Seq workflow using a subsampling Mus musculus data set. Steps in the workflow include read trimming, read QC, read mapping, and counting mapped reads per gene to quantitate gene expression. This tutorial workflow uses truncated and partial run data from the Mittenbühler MJ et al., project.

This tutorial can take over 1 hour 30 minutes to run the code fully. This is part of the reason we have a short and easy introductory tutorial, and this longer more full tutorial for those interested.

![RNA-Seq workflow](../images/rnaseq-workflow.png)

## STEP 1: Install Miniforge

Miniforge is a lightweight Conda distribution that offers a streamlined installation process and efficient package management. It provides access to a vast repository of packages.

Conda packages and environments are useful for several reasons. Conda packages contain metadata. This metadata includes information about what other programs the given software needs to be installed, in order to run. When installing a package with Conda, those other packages are automatically also installed. In this way, the user does not have to worry about manually installing each dependency. This makes installation quick and simple.

These packages are installed inside of environments, which are simply folders within the local installation of Conda. This has several benefits. Local installation means easier installation for non-admin users who may not have access to all system directories. Each environment can hold specific software with specific versions, and it easy to swap to different environments. In addition, the environments themselves are portable, as each environment contains a manifest on how to recreate that environment.

Miniforge itself is a Conda package manager, this means it requires Conda in order to work. It is used to install and update Conda packages, which it gets from a ‘channel’, or repository. It is an alternative to the native Conda package manager. It is often used for reasons of speed.

Bioconda is a ‘channel’, or repository, that the Mambaforge package manager can download packages from. It is a repository of Conda packages that are related to biology. These packages are versions of popular biology software that are curated and uploaded by contributing users.

In [ ]:
# Download Miniforge or Mambaforge (you can use either based on preference)
!curl -L -O https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-$(uname)-$(uname -m).sh

# Install Miniforge (or Mambaforge) - no need to install conda since mamba will be available immediately
!bash Miniforge3-$(uname)-$(uname -m).sh -b -u -p $HOME/miniforge
!date +"%T"

Next, using miniforge and bioconda, install the tools that will be used in this tutorial.

In [ ]:
# Update PATH to point to the Miniforge (or Mambaforge) bin files
import os
os.environ["PATH"] += os.pathsep + os.environ["HOME"]+"/miniforge/bin"

# Now use mamba directly to install your software packages
!mamba install -y -c conda-forge -c bioconda trimmomatic fastqc multiqc sql-magic entrez-direct gffread parallel-fastq-dump sra-tools pyathena samtools star rsem subread pigz -y

## STEP 2: Setup Environment

Create a set of directories to store the reads, reference sequence files, and output files.

In [3]:
!cd $HOMEDIR
!echo $PWD
!mkdir -p data_sub
!mkdir -p data_sub/trunc_rawfastq
!mkdir -p data_sub/trimmed
!mkdir -p data_sub/fastqc_samples/
!mkdir -p data_sub/reference
!mkdir -p data_sub/aligned_bam
!mkdir -p data_sub/rsem_reference/mouse_rsem_reference
!mkdir -p data_sub/rsem_output
!mkdir -p data_sub/reference/STAR_index

/home/ec2-user/SageMaker


Specify the number of available threads based on the VM.

This is useful for later tools such as trimmomatic, or STAR.

In [4]:
import multiprocessing

num_cores = multiprocessing.cpu_count()
THREADS = max(1, num_cores - 1)

print("Number of threads:", THREADS)
os.environ["THREADS"] = str(THREADS)

Number of threads: 15


## STEP 3: Downloading relevant files



### STEP 3.1: Finding run accession numbers.


This code retrieves the SRR IDs associated with project PRJNA892075 from the NCBI database and saves them to a .txt file

In [5]:
!esearch -db sra -query "PRJNA625528" | efetch -format runinfo | cut -d',' -f1 | tail -n +2 > all_accs.txt
!grep -E "SRR11550221|SRR11550223|SRR11550225|SRR11550227|SRR11550229|SRR11550231" all_accs.txt > accs.txt
!cat accs.txt

SRR11550221
SRR11550223
SRR11550225
SRR11550227
SRR11550229
SRR11550231


### STEP 3.2: Download subsampling FASTQ files from S3 bucket

In order for this tutorial to run quickly, we will only analyze 1,000,000 reads from each of the eight samples in PRJNA892075. These subsampled files have been posted on a publicly accessible S3 bucket.

The code will connect to the S3 bucket and download the paired-end FASTQ files for each SRR.

In [6]:
#Load SRR IDs from accs.txt (if available)
with open('accs.txt', 'r') as f:
    accs = [line.strip() for line in f.readlines()]

In [7]:
for acc in accs:
    !wget -P data_sub/raw_fastq/ https://sra-data-athena.s3.amazonaws.com/fastqfiles/subsampled_celegans_{acc}.fastq

--2024-10-10 05:18:00--  https://sra-data-athena.s3.amazonaws.com/fastqfiles/subsampled_celegans_SRR11550221.fastq
Resolving sra-data-athena.s3.amazonaws.com (sra-data-athena.s3.amazonaws.com)... 3.5.27.109, 3.5.29.204, 54.231.166.241, ...
Connecting to sra-data-athena.s3.amazonaws.com (sra-data-athena.s3.amazonaws.com)|3.5.27.109|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 196475402 (187M) [binary/octet-stream]
Saving to: ‘data_sub/raw_fastq/subsampled_celegans_SRR11550221.fastq.1’

subsampled_celegans 100%[===================>] 187.37M  30.1MB/s    in 6.7s    

2024-10-10 05:18:07 (27.9 MB/s) - ‘data_sub/raw_fastq/subsampled_celegans_SRR11550221.fastq.1’ saved [196475402/196475402]

--2024-10-10 05:18:07--  https://sra-data-athena.s3.amazonaws.com/fastqfiles/subsampled_celegans_SRR11550223.fastq
Resolving sra-data-athena.s3.amazonaws.com (sra-data-athena.s3.amazonaws.com)... 52.217.131.33, 3.5.28.195, 52.217.235.97, ...
Connecting to sra-data-athena.s3.a

### STEP 3.3: Download reference genome and annotation files that will be used by STAR and RSEM

This step downloads and unzips reference files for the mouse genome and annotations needed by STAR. These reference files will be used by STAR to align RNA-seq reads during the analysis.

In [8]:
!wget https://sra-data-athena.s3.amazonaws.com/reference/celegans_genome.fa -O data_sub/reference/celegans_genome.fa
!wget https://sra-data-athena.s3.amazonaws.com/reference/celegans_annotation.gtf -O data_sub/reference/celegans_annotation.gtf
!wget https://sra-data-athena.s3.amazonaws.com/reference/celegans_feature_table.txt -O data_sub/reference/celegans_feature_table.txt

--2024-10-10 05:20:28--  https://sra-data-athena.s3.amazonaws.com/reference/celegans_genome.fa
Resolving sra-data-athena.s3.amazonaws.com (sra-data-athena.s3.amazonaws.com)... 54.231.140.233, 52.217.142.105, 54.231.169.145, ...
Connecting to sra-data-athena.s3.amazonaws.com (sra-data-athena.s3.amazonaws.com)|54.231.140.233|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 101957874 (97M) [binary/octet-stream]
Saving to: ‘data_sub/reference/celegans_genome.fa’

data_sub/reference/ 100%[===================>]  97.23M  29.3MB/s    in 3.3s    

2024-10-10 05:20:32 (29.3 MB/s) - ‘data_sub/reference/celegans_genome.fa’ saved [101957874/101957874]

--2024-10-10 05:20:32--  https://sra-data-athena.s3.amazonaws.com/reference/celegans_annotation.gtf
Resolving sra-data-athena.s3.amazonaws.com (sra-data-athena.s3.amazonaws.com)... 52.217.68.20, 52.216.146.211, 52.217.124.241, ...
Connecting to sra-data-athena.s3.amazonaws.com (sra-data-athena.s3.amazonaws.com)|52.217.68.20|:

### STEP 3.4: Copy data file for Trimmomatic


One of trimmomatics functions is to trim sequence machine specific adapter sequences. These are usually within the trimmomatic installation directory in a folder called adapters.

Directories of packages within conda installations can be confusing, so in the case of using conda with trimmomatic, it may be easier to simply download or create a file with the relevant adapter sequencecs and store it in an easy to find directory.

In [9]:
!wget https://sra-data-athena.s3.amazonaws.com/reference/TruSeq3-SE.fa -O data_sub/trimmed/TruSeq3-SE.fa

--2024-10-10 05:20:39--  https://sra-data-athena.s3.amazonaws.com/reference/TruSeq3-SE.fa
Resolving sra-data-athena.s3.amazonaws.com (sra-data-athena.s3.amazonaws.com)... 52.217.228.81, 16.15.216.217, 52.216.137.148, ...
Connecting to sra-data-athena.s3.amazonaws.com (sra-data-athena.s3.amazonaws.com)|52.217.228.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 120 [binary/octet-stream]
Saving to: ‘data_sub/trimmed/TruSeq3-SE.fa’

data_sub/trimmed/Tr 100%[===================>]     120  --.-KB/s    in 0s      

2024-10-10 05:20:39 (8.88 MB/s) - ‘data_sub/trimmed/TruSeq3-SE.fa’ saved [120/120]



### STEP 4: Run Trimmomatic

Trimmomatic will trim off any adapter sequences or low quality sequence it detects in the FASTQ files.

Trimmomatic takes an input, in this case a forward and reverse FASTQ file, and the user names the outputs. In this case the first two outputs are the ‘paired’ and ‘orphaned’ trimmed files for the forward reads. The second two are the ‘paired’ and ‘orphaned’ trimmed files for the reverse reads.

Paired in this case means the forward and reverse sequences were able to be aligned to each other after trimming. Orphaned, or unpaired, typically means one of the reads was discarded as a result of trimming, and only the forward or reverse read survived.

Here we take only the file with paired reads, as there are only a few orphaned reads, and including orphaned reads can complicate different downstream analyses. Unless there is a significant amount of them, or a specific reason to use them, it is generally easier to discard unpaired reads. Also in the interest of simplicity and speed, we proceed in further steps with just using the paired-end reads of just the forward-end read files, which in this context is sufficient, however in different contexts, using both forward and reverse is often preferable.

The last part of the command specifies how the trimming is done: ILLUMINACLIP:TruSeq3-PE.fa:2:30:10:2:keepBothReads LEADING:3 TRAILING:3 MINLEN:36

In greater detail:

‘ILLUMINACLIP:TruSeq3-PE.fa’ refers to which adapters should be cut from the reads.

‘2:30:10:2’ refers to various metrics, which are recommended defaults.

‘2’ refers to the seed mismatch. This refers to the amount of mismatches a 'seed' may have in aligning to a possible adapter.

‘30’ refers to the palindrome clip threshold. This refers to the similarity score. If forward and reverse reads, after having an adapter attached to them, are greater than this score, trimming of adapter fragments will be performed. Forward reads are clipped, and reverse reads dropped.

‘10’ refers to the simple clip threshold. The basic, and alternative method to palindromic searching for adapters. Adapters are tested against reads and if sufficiently matched (above the threshold, in this case, 10), they are clipped.

‘2’ refers to the minimum adapter fragment length in palindrome mode.

‘LEADING:3’ refers to trimming bases from the start of a read. Bases at the start of a read will continue to be trimmed, sequentially, as long as the bases remain below a PHRED score of 3.

‘TRAILING:3’ refers to the same as above, but at the end of a read.

‘MINLEN:36’, the read is discarded if below this length.

Greater information about parameters can be found in the trimmomatic documentation.

In [10]:
!cat accs.txt | xargs -I {} \
trimmomatic SE -threads $THREADS \
data_sub/raw_fastq/subsampled_celegans_{}.fastq data_sub/trimmed/{}_trimmed.fastq \
ILLUMINACLIP:data/trimmed/TruSeq3-SE.fa:2:30:10:2:keepBothReads LEADING:3 TRAILING:3 MINLEN:36

TrimmomaticSE: Started with arguments:
 -threads 15 data_sub/raw_fastq/subsampled_celegans_SRR11550221.fastq data_sub/trimmed/SRR11550221_trimmed.fastq ILLUMINACLIP:data/trimmed/TruSeq3-SE.fa:2:30:10:2:keepBothReads LEADING:3 TRAILING:3 MINLEN:36
Using Long Clipping Sequence: 'AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGTA'
Using Long Clipping Sequence: 'AGATCGGAAGAGCACACGTCTGAACTCCAGTCAC'
ILLUMINACLIP: Using 0 prefix pairs, 2 forward/reverse sequences, 0 forward only sequences, 0 reverse only sequences
Quality encoding detected as phred33
Input Reads: 1000000 Surviving: 1000000 (100.00%) Dropped: 0 (0.00%)
TrimmomaticSE: Completed successfully
TrimmomaticSE: Started with arguments:
 -threads 15 data_sub/raw_fastq/subsampled_celegans_SRR11550223.fastq data_sub/trimmed/SRR11550223_trimmed.fastq ILLUMINACLIP:data/trimmed/TruSeq3-SE.fa:2:30:10:2:keepBothReads LEADING:3 TRAILING:3 MINLEN:36
Using Long Clipping Sequence: 'AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGTA'
Using Long Clipping Sequence: 'AGATCGGAAGAGCA

### STEP 5: Run FastQC
FastQC is an invaluable tool that allows you to evaluate whether there are problems with a set of reads. For example, it will provide a report of whether there is any bias in the sequence composition of the reads.

Because jupyter is at its core a python editor, we can use python code and html support to display results in-line.

FastQC looks for different characteristics of quality in reads. It is very rare that every metric will pass. In many cases, they serve as warnings, which should be compared to the context of the experiment. For instance, here, per base sequence content, sequence length distribution, sequence duplication levels, and overrepresented sequences all throw warnings. Per base sequence content routinely fails in RNA-sequencing in the first 15~ or so bases due to biased fragmentation. In most of our samples, this is where we see the failure (20% or more difference between A/T or G/C), and so is not unexpected. The overrepresented sequences can be BLASTed to show the majority of them are ribosomal RNA. Ribosomal RNA contamination is also common and will not be indexed later, and so not a large concern. Other metrics look good.

In [13]:
# Run FastQC
!cat accs.txt | xargs -P $THREADS -I {} fastqc data_sub/trimmed/{}_trimmed.fastq -o data_sub/fastqc_samples/

null
null
null
null
null
null
Started analysis of SRR11550225_trimmed.fastq
Started analysis of SRR11550231_trimmed.fastq
Started analysis of SRR11550229_trimmed.fastq
Started analysis of SRR11550227_trimmed.fastq
Started analysis of SRR11550221_trimmed.fastq
Started analysis of SRR11550223_trimmed.fastq
Approx 5% complete for SRR11550229_trimmed.fastq
Approx 5% complete for SRR11550231_trimmed.fastq
Approx 5% complete for SRR11550225_trimmed.fastq
Approx 5% complete for SRR11550227_trimmed.fastq
Approx 5% complete for SRR11550223_trimmed.fastq
Approx 5% complete for SRR11550221_trimmed.fastq
Approx 10% complete for SRR11550229_trimmed.fastq
Approx 10% complete for SRR11550231_trimmed.fastq
Approx 10% complete for SRR11550225_trimmed.fastq
Approx 10% complete for SRR11550227_trimmed.fastq
Approx 10% complete for SRR11550221_trimmed.fastq
Approx 10% complete for SRR11550223_trimmed.fastq
Approx 15% complete for SRR11550229_trimmed.fastq
Approx 15% complete for SRR11550231_trimmed.fastq


In [14]:
from IPython.display import IFrame
IFrame(src='./data_sub/fastqc_samples/SRR11550221_trimmed_fastqc.html', width=800, height=600)

### STEP 6: Run MultiQC
MultiQC reads in the FastQC reports and generate a compiled report for all the analyzed FASTQ files.

Being able to use python with bash also means we can seamlessly use popular python packages, such as pandas, to interact with or view the files we create.

In [15]:
#!multiqc -f data_sub/fastqc_samples/
!multiqc -f -o data_sub/multiqc_samples/ data_sub/fastqc_samples/


/// ]8;id=442186;https://multiqc.info\MultiQC]8;;\ 🔍 v1.25.1

       file_search | Search path: /home/ec2-user/SageMaker/data_sub/fastqc_samples
         searching | ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 12/12  lm  
            fastqc | Found 6 reports
     write_results | Data        : data_sub/multiqc_samples/multiqc_data
     write_results | Report      : data_sub/multiqc_samples/multiqc_report.html
           multiqc | MultiQC complete


### RSEM with SLURM

### STEP 7: Download STAR index and rsem-prepare-reference files from S3 Bucket

This step Download from S3 bucket a genome index for STAR (Spliced Transcripts Alignment to a Reference), a highly efficient and accurate aligner optimized for RNA-seq data. STAR is widely recognized for its ability to handle large datasets and accurately map reads across splice junctions, which is critical for eukaryotic RNA-seq analysis where splicing is common.
Also, download the reference index for RSEM (RNA-Seq by Expectation-Maximization), a widely-used tool for quantifying gene and transcript expression from RNA-seq data. The --star and --star-path parameters specify that STAR will be used as the alignment tool, with the path to the STAR executable explicitly provided, allowing RSEM to internally handle the read alignment process by leveraging the STAR index previously generated.
The command processes the provided genome annotation file (.gtf format) and the genome sequence file (.fa format) to build the RSEM reference index.
This step was adapted to be faster to run RSEM. The complete tutorial showin how to download STAR index and rsem-prepare-reference files are in Tutorial 1b.

In [18]:
!rsem-prepare-reference --gtf data_sub/reference/celegans_annotation.gtf --star -p $THREADS data_sub/reference/celegans_genome.fa celegans_reference

rsem-extract-reference-transcripts celegans_reference 0 data_sub/reference/celegans_annotation.gtf None 0 data_sub/reference/celegans_genome.fa
Parsed 200000 lines
Parsed 400000 lines
Parsed 600000 lines
Parsing gtf File is done!
data_sub/reference/celegans_genome.fa is processed!
60140 transcripts are extracted and 0 transcripts are omitted.
Extracting sequences is done!
Group File is generated!
Transcript Information File is generated!
Chromosome List File is generated!
Extracted Sequences File is generated!

rsem-preref celegans_reference.transcripts.fa 1 celegans_reference
Refs.makeRefs finished!
Refs.saveRefs finished!
celegans_reference.idx.fa is generated!
celegans_reference.n2g.idx.fa is generated!

STAR  --runThreadN 15  --runMode genomeGenerate  --genomeDir .  --genomeFastaFiles data_sub/reference/celegans_genome.fa  --sjdbGTFfile data_sub/reference/celegans_annotation.gtf  --sjdbOverhang 100  --outFileNamePrefix celegans_reference
	/home/ec2-user/anaconda3/envs/tensorflow2_p

In [19]:
!STAR  --runThreadN $THREADS  --runMode genomeGenerate  --genomeDir .  --genomeFastaFiles data_sub/reference/celegans_genome.fa  --sjdbGTFfile data_sub/reference/celegans_annotation.gtf  --sjdbOverhang 100  --outFileNamePrefix celegans_reference  --genomeSAindexNbases 12

	/home/ec2-user/anaconda3/envs/tensorflow2_p310/bin/STAR-avx2 --runThreadN 15 --runMode genomeGenerate --genomeDir . --genomeFastaFiles data_sub/reference/celegans_genome.fa --sjdbGTFfile data_sub/reference/celegans_annotation.gtf --sjdbOverhang 100 --outFileNamePrefix celegans_reference --genomeSAindexNbases 12
	STAR version: 2.7.11b   compiled: 2024-07-03T14:39:20+0000 :/opt/conda/conda-bld/star_1720017372352/work/source
Oct 10 05:29:15 ..... started STAR run
Oct 10 05:29:15 ... starting to generate Genome files
Oct 10 05:29:16 ..... processing annotations GTF
Oct 10 05:29:20 ... starting to sort Suffix Array. This may take a long time...
Oct 10 05:29:20 ... sorting Suffix Array chunks and saving them to disk...
Oct 10 05:29:34 ... loading chunks from disk, packing SA...
Oct 10 05:29:36 ... finished generating suffix array
Oct 10 05:29:36 ... generating Suffix Array index
Oct 10 05:29:46 ... completed Suffix Array index
Oct 10 05:29:47 ..... inserting junctions into the genome indice

### STEP 8: Align RNA-Seq Reads with STAR and Quantify Transcript Abundance with RSEM

This step employs the STAR aligner to map RNA-seq reads to the reference genome and generates aligned output in BAM format for each sample. The code processes multiple samples by iterating over a list of Sequence Read Archive (SRA) Run IDs (SRR) specified in the accs.txt file. For each sample, the paired-end FASTQ files are aligned to the reference genome using the STAR index.

The alignment is performed using the --quantMode TranscriptomeSAM GeneCounts option, which directs STAR to not only align the reads but also produce transcriptome-specific BAM files and gene-level read counts. The gene counts output by STAR can be utilized for downstream analyses, such as differential gene expression.

The use of BAM format (--outSAMtype BAM SortedByCoordinate) ensures that the aligned reads are saved in a compressed and indexed format, sorted by genomic coordinate, which is critical for efficient storage and further processing. Paired-end reads are provided to STAR via the --readFilesIn option, and gzip-compressed FASTQ files are handled with the --readFilesCommand zcat argument.

This alignment process typically takes approximately 10 minutes per sample.

In [20]:
import os

# Ensure you've set the path to the RSEM binary
with open('accs.txt', 'r') as f:
    srr_accessions = [line.strip() for line in f.readlines()]
    
# Define the output directory
output_dir = "data_sub/rsem_output"

for srr in srr_accessions:
    os.system(f"rsem-calculate-expression -p $THREADS --star "
              f"data_sub/trimmed/{srr}_trimmed.fastq celegans_reference "
              f"{output_dir}/{srr}celegans")

STAR --genomeDir .  --outSAMunmapped Within  --outFilterType BySJout  --outSAMattributes NH HI AS NM MD  --outFilterMultimapNmax 20  --outFilterMismatchNmax 999  --outFilterMismatchNoverLmax 0.04  --alignIntronMin 20  --alignIntronMax 1000000  --alignMatesGapMax 1000000  --alignSJoverhangMin 8  --alignSJDBoverhangMin 1  --sjdbScore 1  --runThreadN 15  --genomeLoad NoSharedMemory  --outSAMtype BAM Unsorted  --quantMode TranscriptomeSAM  --outSAMheaderHD \@HD VN:1.4 SO:unsorted  --outFileNamePrefix data_sub/rsem_output/SRR11550221celegans.temp/SRR11550221celegans  --readFilesIn data_sub/trimmed/SRR11550221_trimmed.fastq 
	/home/ec2-user/anaconda3/envs/tensorflow2_p310/bin/STAR-avx2 --genomeDir . --outSAMunmapped Within --outFilterType BySJout --outSAMattributes NH HI AS NM MD --outFilterMultimapNmax 20 --outFilterMismatchNmax 999 --outFilterMismatchNoverLmax 0.04 --alignIntronMin 20 --alignIntronMax 1000000 --alignMatesGapMax 1000000 --alignSJoverhangMin 8 --alignSJDBoverhangMin 1 --sjdb

In this part of the code, RSEM utilizes the previously created reference index to map RNA-seq reads to specific genes and accurately estimate gene expression levels. The reference index streamlines the quantification process by precomputing essential data structures, enabling efficient read mapping and abundance estimation.
The code iterates through the SRR IDs, applying RSEM to each sample to quantify gene expression. The effectiveness of RSEM hinges on the reference index, which contains crucial information about the genome and gene annotations. This information guides RSEM in accurately mapping aligned reads and estimating gene expression levels. This step typically takes around 40 minutes to complete.

### STEP 9: Report the top 10 most highly expressed genes in the samples

Top 10 most highly expressed genes in each wild-type sample.


In [21]:
import pandas as pd

# Path to RSEM results directory
rsem_results_dir = 'data/rsem_output'

# Loop through each file in accs.txt
for srr_id in open('accs.txt'):
    srr_id = srr_id.strip()  # Remove newline character
    rsem_result_file = f'{rsem_results_dir}/{srr_id}celegans.genes.results'

    # Load the RSEM results into a Pandas DataFrame
    df = pd.read_csv(rsem_result_file, sep='\t')

    # Sort the DataFrame by TPM values in descending order and get the top 10 genes
    top_10_genes = df.sort_values(by='TPM', ascending=False).head(10)

    # Print the top 10 genes with their TPM values
    print(f"Top 10 Genes by TPM for {srr_id}:")
    print(top_10_genes[['gene_id', 'TPM']])

Top 10 Genes by TPM for SRR11550221:
              gene_id       TPM
3183   WBGene00004622  38022.43
3180   WBGene00004512  26746.09
3181   WBGene00004513  26746.09
3184   WBGene00004677  13321.70
44059  WBGene00201398  11051.05
3113   WBGene00004443   9935.64
4969   WBGene00006650   9872.99
20027  WBGene00023068   9851.86
950    WBGene00001168   6923.95
4779   WBGene00006439   6548.11
Top 10 Genes by TPM for SRR11550223:
              gene_id       TPM
3183   WBGene00004622  47540.45
3180   WBGene00004512  30980.52
3181   WBGene00004513  30980.52
3184   WBGene00004677  16185.31
44059  WBGene00201398  11724.74
20027  WBGene00023068  10710.27
3113   WBGene00004443  10347.68
4969   WBGene00006650   8961.43
950    WBGene00001168   6771.43
4779   WBGene00006439   6316.44
Top 10 Genes by TPM for SRR11550225:
              gene_id       TPM
3183   WBGene00004622  19762.95
3181   WBGene00004513  15963.43
3180   WBGene00004512  15963.43
3113   WBGene00004443  10709.73
20027  WBGene00023068  10

Top 10 most highly expressed genes in the double lysogen samples.


### STEP 10: Report the expression of WBGene00004512 for each file

Use `grep` to report the expression in the wild-type sample. The fields in the RSEM `genes.results` file are as follows. The level of expression is reported in the Transcripts Per Million (`TPM`) and number of reads (`NumReads`) fields:  
`Name    Length  EffectiveLength TPM     NumReads`

In [22]:
import pandas as pd

# Path to RSEM results directory
rsem_results_dir = 'data/rsem_output'

# Target gene ID
target_gene = 'WBGene00004512'

# Loop through each file in accs.txt
for srr_id in open('accs.txt'):
    srr_id = srr_id.strip()  # Remove newline character
    rsem_result_file = f'{rsem_results_dir}/{srr_id}celegans.genes.results'

    # Load the RSEM results into a Pandas DataFrame
    df = pd.read_csv(rsem_result_file, sep='\t')

    # Filter for the target gene
    target_gene_data = df[df['gene_id'] == target_gene]

    # Print the target gene's TPM value for the SRR ID
    print(f"TPM for {target_gene} in {srr_id}: {target_gene_data['TPM'].values[0]}")

TPM for WBGene00004512 in SRR11550221: 26746.09
TPM for WBGene00004512 in SRR11550223: 30980.52
TPM for WBGene00004512 in SRR11550225: 15963.43
TPM for WBGene00004512 in SRR11550227: 55509.8
TPM for WBGene00004512 in SRR11550229: 41323.36
TPM for WBGene00004512 in SRR11550231: 47386.99


## <a name="workflow">Additional Workflows</a>

Now that you have read counts per gene, feel free to explore the R workflow which creates plots and analyses using these readcount files, or try other alternate workflows for creating read count files, such as using snakemake.


[Workflow One (Extended):](Tutorial_1B_Extended_mouse.ipynb) An extended version of workflow one. Once you have got your feet wet, you can retry workflow one with this extended version that covers the entire dataset.

[Workflow Two (DEG Analysis):](Tutorial_2_DEG_Analysis_mouse.ipynb) Using Deseq2 and R to conduct clustering and differential gene expression analysis.

[Workflow Three (Network Analysis):](Tutorial_3_NetAct.ipynb) Using NetAct and R to conduct transcription factor network analysis.
